In [175]:
import pickle
import regex as re
import nltk
import pandas as pd
import numpy as np
import json, os, string
from janitor import clean_names
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from datetime import datetime


In [121]:
def read_json_dict(path: str) -> dict:
    """
    Reads a json file and returns it as dict object
    """
    
    file = open(path) # Opening JSON file
    return json.load(file) # returns JSON object as a dictionary

def folder_creator(folder_name: string, path: string) -> None:
    """
    Generates a folder in specified path
    
    input: name of root folder, path where you want 
    folder to be created
    output: None
    """
    
    # defining paths
    data_folder_path = path + "/" + folder_name
    data_folder_exists = os.path.exists(data_folder_path)

    # creating folders if don't exist
    if data_folder_exists:
        pass
    else:    
        # create a new directory because it does not exist 
        os.makedirs(data_folder_path)

        # create subfolders
        print(f"The new directory {folder_name} was created!")
        
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

def create_pickle(object_name, file_name: str, path: str) -> None:
    """
    Creates a pickle file for object. Note: Path should have no slash 
    at the end
    """
    with open(path + f"/{file_name}", "wb") as storing_output:
        pickle.dump(object_name, storing_output)
        storing_output.close()

## Reading paths

In [3]:
paths = read_json_dict("paths.json")

In [4]:
paths

{'data_path': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice',
 'code_path': '/Users/brandonmora/GitHub/peru-amag-stats/case_outcomes',
 'data_amag_i': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/01_AMAG',
 'data_cej': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/data_cleaned_',
 'data_gender': 'D:/Accesos directos/Trabajo/World Bank/WB Repos/peru-scrape-justice/07_Other/02_Raw/names_gender',
 'local_storage': 'D:/Daniel Chen Dropbox/Marco Antonio GutiÃ©rrez ChÃ¡vez/datasets_amag_ii_scrape'}

In [5]:
data_path = paths["data_path"]

In [6]:
folder_creator("data_cleaned", data_path)

In [7]:
data_cleaned_path = data_path + "/data_cleaned"

In [8]:
folder_creator("raw", data_cleaned_path)

In [9]:
dc_raw_path = data_cleaned_path + "/raw"

In [122]:
folder_creator("temp", data_cleaned_path)

The new directory temp was created!


In [123]:
dc_temp_path = data_cleaned_path + "/temp"

# 1. Creating participants list

Reading lab data

In [11]:
lab_data = pd.read_stata(data_path + "/data/lab_Data/Clean_Full_Data12.dta")

Creating name variables for future fuzzy merge

In [12]:
lab_data["participant_nombre_apellido"] = lab_data["Nombres"] + " " + lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"]
lab_data["participant_nombre_apellido"] = lab_data["participant_nombre_apellido"].str.strip()

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\4041196165.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lab_data["participant_nombre_apellido"] = lab_data["Nombres"] + " " + lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"]


In [13]:
lab_data["participant_apellido_nombre"] = lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"] + " " + lab_data["Nombres"]
lab_data["participant_apellido_nombre"] = lab_data["participant_apellido_nombre"].str.strip()

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\909368064.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  lab_data["participant_apellido_nombre"] = lab_data["ApellidoPaterno"] + " " + lab_data["ApellidoMaterno"] + " " + lab_data["Nombres"]


In [14]:
lab_data = lab_data.rename(columns={"DNI": "nrodocumento"})

In [15]:
amag_ii_participants = lab_data[["nrodocumento", "participant_nombre_apellido", "participant_apellido_nombre"]]

Exporting the list of participants

In [16]:
amag_ii_participants.to_csv(dc_raw_path + "/amag_ii_participants_list.csv")

# 2. Creating Cases List

### 2.0. Selecting reporte files

In [17]:
files_reports = pd.read_csv(dc_raw_path + "/DF_file_report_2022.csv")
files_reports = clean_names(files_reports)

### 2.1. Cleaning the reporte files

Creating lists with characters to be replace

In [18]:
backslash_reps = ["\\(\\*\\)", "\\", "\\([^()]{0,}\\)"]
trailing_and_special_reps = ["^\\s", "\\,", "\\.$", " \\- JUZ$", "\\*"]
other_strs_reps = ["\\- MIXTO Y LIQ", "\\- MIXTO", "\\- JUZ\\. MIXTO", 
                   "- JM", "- INVESTIGACION", "- PAZ LETRADO", "SECOM - ", "- JT"]

### 2.2. Replacing backlashes, special characters and other uninformative characters

In [19]:
empty_reps = backslash_reps + trailing_and_special_reps +  other_strs_reps

In [21]:
for val in empty_reps:    
    files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\3422139958.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\3422139958.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  files_reports["juez_"] = files_reports["juez_"].str.replace(val, "")


In [20]:
name_reps = [["ALFREDO E\\.", "ALFREDO E"], ["BERTHA F\\.", "BERTHA F"], ["CLAUDIO W\\.", "CLAUDIO W"], 
            ["CLAVELITO L\\.", "CLAVELITO L"], ["ELMER L\\.", "ELMER L"], ["ERNESTO A\\.", "ERNESTO A"],
            ["HERBERT M\\.", "HERBERT M"], ["LUZ K\\.", "LUZ K"], ["NANCY S\\.", "NANCY S"], ["JESSICA E\\.", "JESSICA E"],
            ["PATRICIA C\\.", "PATRICIA C"], ["JESSICA P\\.", "JESSICA P"], ["YOLANDA B\\.", "YOLANDA B\\."],
            ["LUZ M\\.", "LUZ M"], ["EDGAR\\.", "EDGAR"], ["C\\. ARTURO", "C ARTURO"], ["ALEXANDER A\\.", "ALEXANDER A"],
            ["RENE G\\.", "RENE G"], ["GUILLERMO S\\.", "GUILLERMO S"], ["FANNY L\\. ",  "FANNY L"], ["ELISA \\(LA", "ELISA"],
            ["JULIA \\(LA", "JULIA"], ["ACEVEDO DIEZ CECILIA", "ACEVEDO DIEZ CECILIA DEL PILAR"], [" J. ", " J "],
            [" K. ", " K "]]

### 2.3. Replacing names with issues

In [22]:
for name_rep in name_reps:
    files_reports["juez_"] = files_reports["juez_"].str.replace(name_rep[0], name_rep[1])

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\4277728364.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  files_reports["juez_"] = files_reports["juez_"].str.replace(name_rep[0], name_rep[1])


### 2.4. Obtaining the names of judges

Some cases have multiple judges assigned to them. As a result, we need to extract these names as we will match the case with the judge information.

In [23]:
files_reports = files_reports[files_reports["juez_"].notna()]

In [24]:
files_reports["juez_splitted"] = files_reports["juez_"].apply(lambda row: row.split("."))

In [25]:
files_reports["n_judges_case"] = files_reports["juez_splitted"].apply(lambda row: len(row))

In [26]:
judge_names = files_reports[files_reports["n_judges_case"] == 1]

In [27]:
multiple_judge_names = files_reports[files_reports["n_judges_case"] != 1]

In [28]:
multiple_judge_names["juez_1"] = multiple_judge_names["juez_splitted"].apply(lambda row: row[0])
multiple_judge_names["juez_2"] = multiple_judge_names["juez_splitted"].apply(lambda row: row[1])
multiple_judge_names["juez_3"] = multiple_judge_names["juez_splitted"].apply(lambda row: row[2])

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\160945290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiple_judge_names["juez_1"] = multiple_judge_names["juez_splitted"].apply(lambda row: row[0])
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\160945290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multiple_judge_names["juez_2"] = multiple_judge_names["juez_splitted"].apply(lambda row: row[1])
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\160945290.py:3: SettingWithCopyWarn

In [30]:
judge_names = judge_names.rename(columns={"juez_": "juez"})

### 2.5. Fuzzy merge with Lab Experiment Data

Creating debuggings dataset to simulate the fuzzy merge

In [42]:
judge_names.columns

Index(['expediente_n°_', 'organo_jurisdiccional_', 'distrito_judicial_',
       'juez', 'especialista_legal_', 'fecha_de_inicio_', 'proceso_',
       'observacion_', 'especialidad_', 'materia_s_', 'estado_',
       'etapa_procesal_', 'fecha_conclusion_', 'ubicacion_',
       'motivo_conclusion_', 'sumilla_', 'juez_splitted', 'n_judges_case'],
      dtype='object')

In [55]:
debug_judges = judge_names[(judge_names["juez"] == "CRUZADO MEJIA MARTIN VALDEMAR") |
                           (judge_names["juez"] == "APAGUEÑO REATEGUI BRYAN ENRIQUE")]

In [57]:
debug_participants = amag_ii_participants[(amag_ii_participants["participant_apellido_nombre"] == "CRUZADO MEJIA MARTIN VALDEMAR") |
                                          (amag_ii_participants["participant_apellido_nombre"] == "APAGUEÑO REATEGUI BRYAN ENRIQUE")]

Fuzzy match of cases

In [66]:
matched_judge_name1 = fuzzy_merge(debug_judges, debug_participants, "juez", "participant_apellido_nombre", threshold=90, limit=2)

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\3876909639.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\3876909639.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['matches'] = m2


In [132]:
amag_ii_cases = matched_judge_name1.copy()

# 3. Creating CEJ datasets

A pending task would be to bind the rows of all the dataframes

## Follow up dataframe

In [38]:
files_follow_up = pd.read_csv(dc_raw_path + "/DF_follow_up_cleaner_2022.csv", error_bad_lines=False)
files_follow_up = clean_names(files_follow_up)

C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\994655769.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  files_follow_up = pd.read_csv(dc_raw_path + "/DF_follow_up_cleaner_2022.csv", error_bad_lines=False)
Skipping line 8128: expected 10 fields, saw 11



# Procedural parts dataframe

In [112]:
files_procedural_parts = pd.read_csv(dc_raw_path + "/DF_procedural_parts_2022.csv")
files_procedural_parts = clean_names(files_procedural_parts)

In [113]:
files_procedural_parts["expediente_n°_"] = files_procedural_parts["expediente_n°_"].apply(lambda row: row.split("\\")[-1])

## Downloads dataframe

In [40]:
files_downloads = pd.read_csv(dc_raw_path + "/DF_DOWNLOADS_2022.csv")
files_downloads = clean_names(files_downloads)

### Merging complementary case data with amag ii cases

In [133]:
reportes_amag_ii_raw = pd.merge(amag_ii_cases, files_reports, how="inner", on="expediente_n°_")

In [134]:
follow_up_amag_ii_raw = pd.merge(amag_ii_cases, files_follow_up, how="inner", on="expediente_n°_")

In [135]:
procedural_parts_amag_ii_raw = pd.merge(amag_ii_cases, files_procedural_parts, how="inner", on="expediente_n°_")

In [136]:
downloads_amag_ii_raw = pd.merge(amag_ii_cases, files_downloads, how="inner", left_on="expediente_n°_", right_on="expediente_num")

In [125]:
help(create_pickle)

Help on function create_pickle in module __main__:

create_pickle(object_name, file_name: str, path: str) -> None
    Creates a pickle file for object. Note: Path should have no slash 
    at the end



Storing data on `.pkl` files

In [215]:
create_pickle(amag_ii_cases, "amag_ii_cases.pkl", dc_temp_path)

In [216]:
create_pickle(reportes_amag_ii_raw, "reportes_amag_ii_raw.pkl", dc_temp_path)

In [217]:
create_pickle(follow_up_amag_ii_raw, "follow_up_amag_ii_raw.pkl", dc_temp_path)

In [218]:
create_pickle(procedural_parts_amag_ii_raw, "procedural_parts_amag_ii_raw.pkl", dc_temp_path)

In [219]:
create_pickle(downloads_amag_ii_raw, "downloads_amag_ii_raw.pkl", dc_temp_path)

# 4. Preprocessing and cleaning of datasets

Reading gender dataframe

In [130]:
gender_dataset = pd.read_csv(dc_raw_path + "/harvard_set_gender.csv")

In [159]:
def spanish_cleaner(txt_file):
    text = txt_file
    text = re.sub(r"(&[a-zA-Z]*;)", " ", text)  # the txt files had some unwanted text like &rsquo; this line removes such text
    text = text.lower()

    # remove punctuation and numbers from the string
    punctuations = '''!()[]{};:'"\,<>./¿?@#$%^&*_–~=+¨`“”’|0123456789'''  # all but hyphens
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # replacing encoding characters
    enc_characters = [" st ", " nd ", " rd ", " th ", "srl", "lpfvf", "pctc", "jmxcff", "ayrq", "axu", "oadk", "jcxj", "nplt", "eef", "fcfc", "qyoc", "gobpe", "pfg", "vqrx", "csjppj", "xas", "feeback", "hafceqc", "xqj", "hellip", "rsquo", "ldquo", "rdquo", "ndash", "-", "n°", "nº", "º", "°", "dprgdonpdl", "«", "»", "…", "derjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderjudicialpoderj", "ii", "iii", "vii", "viii"]
    
    for item in enc_characters:
        text = text.replace(item, " ")
    
    # cleaning for spanish stop words
    stopword_es = nltk.corpus.stopwords.words('spanish') # loading spanish stop words
    custom_substrs = ["http", "hangouts", "meet", "gmailcom"] # html related
    custom_gender_words = ["él", "ella", "la", "ese", "esa", "esos", "esas", "este", "esta", "aquel", "aquella", "aquellos", "aquellas", "lo", "la", "los", "las", "aquel", "aquella", "mío", "mía", "míos", "mías", "suyo", "suya", "suyos", "suyas"] # list with pronouns associated to a specific gender
    length_custom_stopwords = len(custom_substrs)
    words = text.split() # tokenizing sentence
    cleaned_words = [word for word in words if (word not in stopword_es and len(word) > 1) or word in custom_gender_words]
     
    sentence_no_custom = [] # omitting words that contain 
    for cleaned_word in cleaned_words:
        counter_stopwords = 0
        for word in custom_substrs: # evaluating if word contains substr
            if word not in cleaned_word: # if passes, +1 for counter
                counter_stopwords += 1
            if counter_stopwords == length_custom_stopwords: # append if passes all custom substrs tests
                sentence_no_custom.append(cleaned_word)

    return " ".join(sentence_no_custom)

<>:7: DeprecationWarning: invalid escape sequence '\,'
<>:7: DeprecationWarning: invalid escape sequence '\,'
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\3245748647.py:7: DeprecationWarning: invalid escape sequence '\,'
  punctuations = '''!()[]{};:'"\,<>./¿?@#$%^&*_–~=+¨`“”’|0123456789'''  # all but hyphens


## 4.1. Preprocessing of downloads and follow up 

### Downloads Dataframe

In [147]:
# lowercase to text
downloads_amag_ii_raw["text"] = downloads_amag_ii_raw["text"].apply(lambda text: text.lower() 
                                                                    if type(text) is str else text)

In [161]:
# cleaning the text from the cases
downloads_amag_ii_raw["text"] = downloads_amag_ii_raw["text"].apply(lambda text: spanish_cleaner(text) 
                                                                    if type(text) is str else text)

### Downloads Follow Up

Fixing the date of the resolution

In [196]:
follow_up_amag_ii_raw["fecha_de_resolucion_ingreso_"] = follow_up_amag_ii_raw["fecha_de_resolucion_ingreso_"].apply(
                                                        lambda date: datetime.strptime(re.match("(\d+[-/]\d+[-/]\d+)", 
                                                        date)[0], "%d/%m/%Y"))

<>:2: DeprecationWarning: invalid escape sequence '\d'
<>:2: DeprecationWarning: invalid escape sequence '\d'
C:\Users\Gigabyte\AppData\Local\Temp\ipykernel_68832\1630532510.py:2: DeprecationWarning: invalid escape sequence '\d'
  lambda date: datetime.strptime(re.match("(\d+[-/]\d+[-/]\d+)",


Remove extra white space and lower "acto"

In [200]:
# lowercase to text
follow_up_amag_ii_raw["acto_"] = follow_up_amag_ii_raw["acto_"].apply(lambda text: text.lower() 
                                                                    if type(text) is str else text)

In [201]:
# cleaning the text from the cases
follow_up_amag_ii_raw["acto_"] = follow_up_amag_ii_raw["acto_"].apply(lambda text: spanish_cleaner(text) 
                                                                    if type(text) is str else text)

Remove extra white space and lower "sumilla" (from merged dataframes)

In [207]:
sumillas = ["sumilla__x", "sumilla__y"]

In [205]:
for sumilla in sumillas:
    # lowercase to text
    follow_up_amag_ii_raw[sumilla] = follow_up_amag_ii_raw[sumilla].apply(lambda text: text.lower() 
                                                                    if type(text) is str else text)
    # cleaning the text from the cases
    follow_up_amag_ii_raw[sumilla] = follow_up_amag_ii_raw[sumilla].apply(lambda text: spanish_cleaner(text)
                                                                    if type(text) is str else text)

Remove extra white space and lower "descripcion de usuario"

In [211]:
# lowercase to text
follow_up_amag_ii_raw["descripcion_de_usuario_"] = follow_up_amag_ii_raw["descripcion_de_usuario_"].apply(lambda text: text.lower() 
                                                                    if type(text) is str else text)

In [212]:
# cleaning the text from the cases
follow_up_amag_ii_raw["descripcion_de_usuario_"] = follow_up_amag_ii_raw["descripcion_de_usuario_"].apply(lambda text: spanish_cleaner(text) 
                                                                    if type(text) is str else text)

Create variable that identifies rows with pdf or docx file in downloads dataset

In [213]:
follow_up_amag_ii_raw["descargado"] = follow_up_amag_ii_raw["descripcion_de_usuario_"].apply(lambda text: 1
                                                                                            if "descargado" in text
                                                                                            else 0)